In [1]:
# Import Library
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Layer 
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import  CSVLogger
from tensorflow.keras.callbacks import  ReduceLROnPlateau
from tensorflow.keras.callbacks import  EarlyStopping

In [2]:
# Extra Learnable 
class ClassToken(Layer):
    def __init__(self,):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(1, 1, input_shape[-1]), dtype="float32"),
            trainable=True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)

        return cls

In [3]:
# MultiLinear Perceptron 
# Artificial Neural Network
def mlp(x, cf):
    x = Dense(cf["mlp_dim"], activation="relu")(x)
    x = Dropout(cf["dropout_rate"])(x)
    x = Dense(cf["hidden_dim"], activation="relu")(x)
    x = Dropout(cf["dropout_rate"])(x)
    return x

In [4]:
# Transformer Encoder
def transformer_encoder(x, cf):
    skip_1 = x
    x = LayerNormalization()(x)
    x = MultiHeadAttention(num_heads=cf["num_heads"], key_dim=cf["hidden_dim"])(x, x)
    x = Add()([x, skip_1])

    skip_2 = x
    x = LayerNormalization()(x)
    x = mlp(x, cf)
    x = Add()([x, skip_2])

    return x

In [5]:
config = {}
config["num_layers"] = 1
config["hidden_dim"] = 32
config["mlp_dim"] = 512
config["num_heads"] = 1
config["dropout_rate"] = 0.25

config["image_size"] = 224
config["patch_size"] = 32
config["num_patches"] = int(config["image_size"]**2 / config["patch_size"]**2)
config["num_channels"] = 3
config["num_classes"] = 8
cf = config

In [9]:
# Build the Model
from tensorflow.keras.applications import ResNet50
inputs = Input((224,224,3)) ## (None, 224, 224, 3)
resnet50 = ResNet50(include_top=False, weights="imagenet", input_tensor=inputs)
#output = resnet50.output ## (None, 7, 7, 2048)

# Add custom layers on top of the base model
x = resnet50.output
print("Output shape : ",x.shape)
patch_embed = Conv2D(32,kernel_size=3,padding="same")(x)
x = GlobalAveragePooling2D()(patch_embed)
print("patch_embed : ",patch_embed.shape) ## (None, 7, 7, 768)

Output shape :  (None, 7, 7, 2048)
patch_embed :  (None, 7, 7, 32)


In [10]:
_, h, w, f = patch_embed.shape
patch_embed = Reshape((h*w, f))(patch_embed) ## (None, 49, 768)


""" Position Embeddings """
positions = tf.range(start=0, limit=cf["num_patches"], delta=1) ## (49,)
pos_embed = Embedding(input_dim=cf["num_patches"], output_dim=cf["hidden_dim"])(positions)

""" Patch + Position Embeddings """
embed = patch_embed + pos_embed ## (None, 49, 768)

""" Adding Class Token """
token = ClassToken()(embed)
x = Concatenate(axis=1)([token, embed]) ## (None, 50, 768)

""" Transformer Encoder """
for _ in range(cf["num_layers"]):
    x = transformer_encoder(x, cf)

x = LayerNormalization()(x)
x = x[:, 0, :]
x = Dense(cf["num_classes"], activation="softmax")(x)
model = Model(inputs, x)

In [11]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_pad (ZeroPadding2D)     │ (None, 230, 230, 3)       │               0 │ input_layer_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, 112, 112, 64)      │           9,472 │ conv1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, 112, 112, 64)      │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, 112, 112, 64)      │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pad (ZeroPadding2D)     │ (None, 114, 114, 64)      │               0 │ conv1_relu[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pool (MaxPooling2D)     │ (None, 56, 56, 64)        │               0 │ pool1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, 56, 56, 64)        │           4,160 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, 56, 56, 64)        │          36,928 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_2_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_2_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_3_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ conv2_block1_2_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 24,215,560 (92.38 MB)

 Trainable params: 24,162,440 (92.17 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [12]:
# Read the DataSet For Training And Testing
# Pre-processing For Training Data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True)

# Pre-processing For test Data
test_datagen  = ImageDataGenerator(rescale = 1./255)

# Training data
train_set = train_datagen.flow_from_directory('D:/Copy_AffectNet/train_set',
                                                target_size=(224,224),
                                                batch_size=32,
                                                class_mode='categorical')


# Test data
test_set = test_datagen.flow_from_directory('D:/Copy_AffectNet/test_set',
                                                target_size=(224,224),
                                                batch_size=32,
                                                class_mode='categorical')

Found 38021 images belonging to 8 classes.
Found 3999 images belonging to 8 classes.


In [13]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(0.001, clipvalue=1.0),
    metrics=["acc"])

In [14]:
import os
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
create_dir("files")

In [15]:
model_path = os.path.join("files", "model.keras")
csv_path = os.path.join("files", "log.csv")

In [16]:
callbacks = [
    ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-10, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False)]

In [ ]:
model.fit(
    train_set,
    epochs=50,
    validation_data=test_set,
    callbacks=callbacks
)

Epoch 1/50


C:\Users\Mohamed_Abdeldayem\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1189/1189 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - acc: 0.4074 - loss: 1.6433
Epoch 1: val_loss improved from inf to 2.06549, saving model to files\model.keras
1189/1189 ━━━━━━━━━━━━━━━━━━━━ 8817s 7s/step - acc: 0.4074 - loss: 1.6432 - val_acc: 0.2418 - val_loss: 2.0655 - learning_rate: 0.0010
Epoch 2/50
1189/1189 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - acc: 0.5143 - loss: 1.3254
Epoch 2: val_loss improved from 2.06549 to 1.78033, saving model to files\model.keras
1189/1189 ━━━━━━━━━━━━━━━━━━━━ 7882s 7s/step - acc: 0.5143 - loss: 1.3254 - val_acc: 0.3518 - val_loss: 1.7803 - learning_rate: 0.0010
Epoch 3/50
1189/1189 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - acc: 0.5566 - loss: 1.2190
Epoch 3: val_loss improved from 1.78033 to 1.68688, saving model to files\model.keras
1189/1189 ━━━━━━━━━━━━━━━━━━━━ 7760s 7s/step - acc: 0.5566 - loss: 1.2190 - val_acc: 0.3821 - val_loss: 1.6869 - learning_rate: 0.0010
Epoch 4/50
 899/1189 ━━━━━━━━━━━━━━━━━━━━ 32:00 7s/step - acc: 0.5770 - loss: 1.1596

In [ ]:
#Plot The Training And Testing
# summarize history for accuracy
import matplotlib.pyplot as plt
plt.figure(1)
plt.subplot(211)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.ylim([0.75,1])
plt.legend(['train', 'test'], loc='lower right')
plt.savefig('plot1')

In [ ]:
# summarize history for loss
plt.subplot(212)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.ylim([0,15])
plt.legend(['train', 'test'], loc='upper right')
plt.savefig('plot2')